# Renewable Ressources

> Class for environment with Renewable Ressources

This code is an adaptation to pyCLRD of the code used in this article: [https://doi.org/10.1103/PhysRevE.105.034409](https://doi.org/10.1103/PhysRevE.105.034409)

In [ ]:
#| default_exp Environments/RenewableRessources

In [ ]:
#| hide
# Imports for the nbdev development environment
from nbdev.showdoc import *

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

## Implementation

In [ ]:
#| export
from pyCRLD.Environments.Base import ebase
from pyCRLD.Utils.Helpers import make_variable_vector

from fastcore.utils import *
from fastcore.test import *

from typing import Iterable
import numpy as np
from scipy.stats import norm

In [ ]:
#| export
class RenewableRessources(ebase):
    """
    Environment with Renewable Ressources.
    """ 
    def __init__(self, r, C, pR=0.1, obs=None, deltaE=0.2, sig=1.0):
        self.r = r    # regrowth rate
        self.C = C    # Capacity
        self.pR = pR  # recovery propbability in the case of depeletion

        self.N = 1  # starting with one agent, but this could be made adaptive
        self.M = 3  # 2 for now, but eventually three?
        self.Z = len(self._growth_dict())
        
        self.obs = obs
        
        self.dE = deltaE  # difference from max_sus_yield form low and high 
        self.sig = sig  # std of normal for state transitions
        
        self.T = self.TransitionTensor()
        self.R = self.RewardTensor()
        self.O = self.ObservationTensor()
        super().__init__()

In [ ]:
#| export
@patch
def _growth(self:RenewableRessources, stock):
    return self.r * stock * (1 - stock / self.C)


In [ ]:
#| export
@patch
def _growth_dict(self:RenewableRessources):
    gdic = {0: self._growth(0)}

    stock = 1
    while self._growth(stock) > 0:
        gdic[stock] = self._growth(stock)
        stock += 1

    return gdic


In [ ]:
#| export
@patch
def _action_values(self:RenewableRessources):
    """
    What are the extraction levels corresponding to actions?
    TODO: To be adjusted when multi agent system is considered.
    """
    gdic = self._growth_dict()
    max_sus_yield = max(gdic.values())
    zer_extract = 0
    low_extract = (1-self.dE) * max_sus_yield
    hig_extract = (1+self.dE) * max_sus_yield
    return zer_extract, low_extract, hig_extract

In [ ]:
#| export
@patch
def actions(self:RenewableRessources):
    z, l, h = self._action_values()
    return [0, 1, 2], ["0","low","high"]


In [ ]:
#| export
@patch
def states(self:RenewableRessources):
    return [i for i in range(len(self._growth_dict()))]

In [ ]:
#| export
@patch

def obs_action_space(self:RenewableRessources):
    return np.zeros((self.Q, self.M))


In [ ]:
#| export
@patch
def TransitionTensor(self:RenewableRessources):
    """Get the Transition Tensor."""
    dim = np.concatenate(([self.Z],
                          [self.M for _ in range(self.N)],
                          [self.Z]))
    Tsas = np.ones(dim) * (-1)

    for index, _ in np.ndenumerate(Tsas):
        Tsas[index] = self._transition_probability(index[0],
                                                   index[1:-1],
                                                   index[-1])
    return Tsas

In [ ]:
#| export
@patch
def _transition_probability(self:RenewableRessources, s, jA, sprim):
    acts = np.array(jA)
    act_vals = np.array(self._action_values())

    total_harvest = sum(act_vals[acts])
    harvest_stock = max(s - total_harvest, 0)
    new_stock = max(harvest_stock + self._growth(harvest_stock),
                    self._recoverP(jA))
    new_stock = min(new_stock, self.Z-1)

    # lower_state = int(new_stock)
    # upper_state = lower_state+1
    # uniform distribution between neigboring states
    # if sprim == lower_state:
    #     p = upper_state - new_stock
    # elif sprim == upper_state:
    #     p = new_stock - lower_state
    # else:
    #     p = 0
        
    # gaussian distribution with std `sig` around new_stock
    sig = self.sig
    
    if sprim == 0:  # minimum 
        p = norm.cdf(0.5, new_stock, sig)
    elif sprim == self.Z-1: # maximum
        p = 1 - norm.cdf(self.Z-1.5, new_stock, sig)
    else:
        p = norm.cdf(sprim+0.5, new_stock, sig)\
            - norm.cdf(sprim-0.5, new_stock, sig)
         
    return p

The `TransitionTensor` is obtained with the help of the `_transition_probability` method.

In [ ]:
show_doc(RenewableRessources._transition_probability)

---

[source](https://github.com/wbarfuss/pyCRLD/blob/main/pyCRLD/Environments/RenewableRessources.py#L112){target="_blank" style="float:right; font-size:smaller"}

### RenewableRessources._transition_probability

>      RenewableRessources._transition_probability (s, jA, sprim)

In [ ]:
#| export
@patch

def _recoverP(self:RenewableRessources, jA):
    '''
    makes random recovery action dependent.
    It must pay of to choose low at degredation
    '''
    hig_recoverP = (1+self.dE) * self.pR
    low_recoverP = (1-self.dE) * self.pR
    zer_recoverP = 0
    
    recover_vals = np.array([hig_recoverP, low_recoverP, zer_recoverP])
    
    return recover_vals[jA].mean()

In [ ]:
#| export
@patch

def RewardTensor(self:RenewableRessources):
    """Get the Reward Tensor R[i,s,a1,...,aN,s']."""
    dim = np.concatenate(([self.N],
                          [self.Z],
                          [self.M for _ in range(self.N)],
                          [self.Z]))
    Risas = np.zeros(dim)

    for index, _ in np.ndenumerate(Risas):
        Risas[index] = self._reward(index[0], index[1], index[2:-1],
                                    index[-1])
    return Risas

In [ ]:

#| export
@patch

def ObservationTensor(self:RenewableRessources):
    
    if self.obs is None:
        self.obs = [[s] for s in range(self.Z)]
    self.Q = len(self.obs)
    
    dim = np.concatenate(([self.N],
                  [self.Z],
                  [self.Q]))
    Oiso = np.zeros(dim)

    for o in range(self.Q):
        for s in self.obs[o]:
            Oiso[:,s,o] = 1
            
    Oiso = Oiso / Oiso.sum(-1, keepdims=True)

    return Oiso

The `RewardTensor` is obtained with the help of the `_reward` method.

In [ ]:
#| export
@patch

def _reward(self:RenewableRessources, i, s, jA, sprim):
    act_vals = np.array(self._action_values())
    reward = 0.1*act_vals[jA[i]] if s == 0 or sprim == 0\
        else act_vals[jA[i]]
    return reward

In [ ]:
show_doc(RenewableRessources._reward)

---

[source](https://github.com/wbarfuss/pyCRLD/blob/main/pyCRLD/Environments/RenewableRessources.py#L220){target="_blank" style="float:right; font-size:smaller"}

### RenewableRessources._reward

>      RenewableRessources._reward (i, s, jA, sprim)

In [ ]:
#| export
@patch
def id(self:RenewableRessources):
    """
    Returns id string of environment TODO
    """
    # Default
    def shorten(a): 
        return a
        
    r= shorten(self.r)
    C= shorten(self.C)

    id = f"{self.__class__.__name__}_"+\
        f"{self.N}_{str(r)}_{str(C)}"

    return id

# Example

In [ ]:
#| export
from pyCRLD.Environments.RenewableRessources import RenewableRessources
from pyCRLD.Agents.POStrategyActorCritic import POstratAC
import numpy as np

We will show the effect witnessed in the article: limited information can lead to better strategies. 

In [ ]:
# We will use the parameters from POLD on github, this function is 

def random_behavior(selfi, method="norm"):
    """Behavior profile with random probabilities."""
    if method=="norm":
        X = np.random.rand(selfi.N, selfi.Q, selfi.M)
        X = X / X.sum(axis=2).repeat(selfi.M).reshape(selfi.N, selfi.Q,
                                                     selfi.M)
    elif method == "diff":
        X = np.random.rand(selfi.N, selfi.Q, selfi.M-1)
        X = np.concatenate((np.zeros((selfi.N, selfi.Q, 1)),
                            np.sort(X, axis=-1),
                            np.ones((selfi.N, selfi.Q, 1))), axis=-1)
        X = X[:, :, 1:] - X[:, :, :-1]
    return X
def random_reward(env,test):
    X = np.array(random_behavior(env))
    xtraj, fixedpointreached = test.trajectory(X)
    States = test.Ps(X)
    Rewards = test.Rio(xtraj[-1])[0]
    n = len(States)
    reward = sum([ States[k]*Rewards[k] for k in range(n)])
    return reward

### Same environment with different observability for agents

In the first environment, ```obs = None``` is a shortcut to say that all environment states are observable clearly. In the two others the observations are specified. We can see that limited observation can lead to better reward.

In [ ]:
env = RenewableRessources(r=0.8, C=8, pR=0.1, obs=None, deltaE=0.2, sig=0.5)
test = POstratAC(env=env, learning_rates=0.02, discount_factors=0.9, choice_intensities= 250)
L = [] 
for k in range(100):
    L.append(random_reward(env,test))
print(np.mean(L))

1.2267892


In [ ]:
env = RenewableRessources(r=0.8, C=8, pR=0.1, obs=[[0,1],[2,3,4],[5],[6],[7]], deltaE=0.2, sig=0.5)
test = POstratAC(env=env, learning_rates=0.02, discount_factors=0.9, choice_intensities= 250)
L = [] 
for k in range(100):
    L.append(random_reward(env,test))
print(np.mean(L))

1.2735934


In [ ]:
env = RenewableRessources(r=0.8, C=8, pR=0.1, obs=[[0,1,2,3,4],[5,6,7]], deltaE=0.2, sig=0.5)
test = POstratAC(env=env, learning_rates=0.02, discount_factors=0.9, choice_intensities= 250)
L = [] 
for k in range(100):
    L.append(random_reward(env,test))
print(np.mean(L))

0.30748004
